In [ ]:
from manim import *
import numpy as np
from manim import TexTemplate

# 中文TeX模板设置
my_template = TexTemplate()
my_template.add_to_preamble(r"\usepackage{ctex}")

class SimpleLinearRegression(Scene):
    def construct(self):
        # 设置随机种子
        np.random.seed(42)
        
        # 创建随机数据点
        x_vals = np.linspace(1, 5, 8)
        y_vals = 2 * x_vals + 1 + np.random.normal(0, 1.9, len(x_vals))
        y_vals = np.maximum(y_vals, 0.5)
        
        # 创建坐标轴
        axes = Axes(
            x_range=[0, 6, 1],
            y_range=[0, 15, 3],
            x_length=5,
            y_length=5,
            axis_config={"color": BLUE},
        ).shift(LEFT*0.5)
        axes_labels = axes.get_axis_labels(x_label="x", y_label="y")
        
        # 创建数据点
        dots = VGroup(*[
            Dot(axes.coords_to_point(x, y), 
                color="#FF6347",
                radius=0.1,
                fill_opacity=0.9
            ) for x, y in zip(x_vals, y_vals)
        ])
        
        # 计算最佳拟合线
        A = np.vstack([x_vals, np.ones(len(x_vals))]).T
        m, c = np.linalg.lstsq(A, y_vals, rcond=None)[0]
        best_fit_line = axes.plot(
            lambda x: m * x + c,
            color="#4682B4",
            stroke_width=5
        )
        
        # 动画序列
        self.play(Create(axes), Write(axes_labels), run_time=1.5)
        self.wait(0.5)
        
        for i, dot in enumerate(dots):
            self.play(
                dot.animate.scale(1.5).set_color("#FF4500"),
                rate_func=there_and_back_with_pause,
                run_time=0.1
            )
            if i < len(dots) - 1:
                self.wait(0.1)
        self.wait(0.5)
        
        self.play(Create(best_fit_line), run_time=2.5)
        self.wait(1)
        
        # 移动坐标系到左侧
        original_system = VGroup(axes, axes_labels, dots, best_fit_line)
        self.play(original_system.animate.shift(LEFT*3), run_time=1.5)
        self.wait(1)
        
        # 创建四个小坐标系
        small_systems = VGroup()
        variations = [
            (0.9*m, 1.1*c),
            (1.1*m, 0.9*c),
            (0.8*m, 1.2*c),
            (1.2*m, 0.8*c)
        ]
        
        # 田字格布局
        for i in range(2):
            for j in range(2):
                small_axes = Axes(
                    x_range=[0, 6, 1],
                    y_range=[0, 15, 3],
                    x_length=2.5,
                    y_length=2.5,
                    axis_config={"color": BLUE, "stroke_width": 2}
                )
                
                small_axes.move_to(RIGHT*1.7 + UP*0.7+ UP*(1 - i*3.0) + RIGHT*(j*3.2))
                
                small_labels = small_axes.get_axis_labels(
                    x_label="x", 
                    y_label="y"
                ).scale(0.7)
                
                small_dots = VGroup(*[
                    Dot(small_axes.coords_to_point(x, y), 
                        color="#FF6347",
                        radius=0.06,
                        fill_opacity=0.9
                    ) for x, y in zip(x_vals, y_vals)
                ])
                
                var_m, var_c = variations[i*2 + j]
                small_line = small_axes.plot(
                    lambda x: var_m * x + var_c,
                    color="#4682B4",
                    stroke_width=3
                )
                
                small_system = VGroup(small_axes, small_labels, small_dots, small_line)
                small_systems.add(small_system)
        
        # 显示小坐标系
        self.play(LaggedStart(*[
            Create(small_system) for small_system in small_systems
        ], lag_ratio=0.3), run_time=3)
        
        self.wait(3)

        # 移除小坐标系
        self.play(FadeOut(small_systems), run_time=1)
        self.wait(0.5)

        # 添加误差线
        vertical_lines = VGroup()
        for x, y in zip(x_vals, y_vals):
            y_fit = m * x + c
            
            line = DashedLine(
                start=axes.coords_to_point(x, y),
                end=axes.coords_to_point(x, y_fit),
                dash_length=0.09,
                dashed_ratio=0.7,
                color=YELLOW,
                stroke_width=3
            )
            vertical_lines.add(line)
        
        # 显示误差线
        self.play(LaggedStart(*[
            Create(line) for line in vertical_lines
        ], lag_ratio=0.2), run_time=2)
        
        self.wait(1)

        # 创建误差公式
        part1 = MathTex(r"\text{总误差} = \Sigma(\Delta y)", tex_template=my_template)
        part2 = MathTex(r"= \Sigma(|y - y_{\text{预测}}|)", tex_template=my_template)

        formula = VGroup(part1, part2).arrange(DOWN, aligned_edge=LEFT)
        part1.set_color_by_tex("总误差", GREEN_C)
        part2.set_color_by_tex("实际", GREEN_C)
        part2.set_color_by_tex("预测", GREEN_C)

        formula.next_to(original_system, RIGHT, buff=1.5).shift(UP*1.8)

        self.play(Write(part1), run_time=1)
        self.wait(0.5)
        self.play(Write(part2), run_time=1.5) 
        self.wait(2)
        
        # 添加箭头
        arrow = Arrow(
            start=formula.get_bottom(),
            end=formula.get_bottom() + DOWN*0.8,
            color=YELLOW,
            buff=0.1,
            stroke_width=14,
            tip_length=0.2
        )
        
        # 最小二乘公式
        min_formula = MathTex(
            r"\text{min(} \ \Sigma(y - \hat{y})^2)",
            tex_template=my_template
        ).next_to(arrow, DOWN)
        min_formula.set_color_by_tex("min", BLUE)
        min_formula.set_color_by_tex("Sigma", RED)
        
        self.play(
            GrowArrow(arrow),
            run_time=1
        )
        self.play(
            Write(min_formula),
            run_time=1.5
        )
        self.wait(2)
        
        # 只保留最小二乘公式
        to_fade_out = [original_system, vertical_lines, formula, arrow]
        self.play(
            *[FadeOut(obj) for obj in to_fade_out],
            run_time=1.5
        )
        
        # 显示"最小二乘法"标题
        title = Tex("最小二乘法", tex_template=my_template)
        title.scale(1.9).move_to(ORIGIN).shift(LEFT*2.7 + UP *2.5)
        title.set_color(RED)
        
        self.play(Write(title), run_time=1)
        
        # 移动公式到标题右侧
        min_formula.generate_target()
        min_formula.target.scale(1.3).next_to(title, RIGHT, buff=0.8)
        
        self.play(
            MoveToTarget(min_formula),
            run_time=1.0
        )

        self.wait(3)